In [1]:
import csv
from csv import writer
import pandas as pd

In [2]:
from openai import OpenAI

In [3]:
# The contents of this cell are adopted from
# https://github.com/UG-Team-Data-Science/ai-in-edu/blob/main/utils.py
# created by Michiel van der Ree.

openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"
model = "meta-llama/Meta-Llama-3-8B-Instruct"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

def format_history(message, history, system_prompt):
    chat_history = [{"role": "system", "content": system_prompt}]
    for query, response in history:
        chat_history.append({"role": "user", "content": query})
        chat_history.append({"role": "assistant", "content": response})
    chat_history.append({"role": "user", "content": message})
    return chat_history

def prompt_model(
    message,
    history=None,
    system_prompt="You are a powerful annotator of event triggers and non-event triggers",
):
    chat_history = format_history(message, history if history else [], system_prompt)
    response = client.chat.completions.create(model=model, messages=chat_history)
    return response.choices[0].message.content

In [2]:
df1 = pd.read_csv('TimeML_TEST_list_of_sentences.csv')

In [3]:
df1.head(10)

,Unnamed: 0,sentences
0,0,"A Cyprus exit from the euro union, if it come..."
1,1,"And for European unity and diplomacy, the Cypr..."
2,2,"ut for the broader financial system in Europe,..."
3,3,"And that, economists and investors contend, is..."
4,4,Greece may well have been too big to fail last...
5,5,"From a financial standpoint, what is most note..."
6,6,"Only Ireland, still struggling to recover from..."
7,7,As debts in Europe mount in inverse proportion...
8,8,`` There have been too many bailouts in Europe...
9,9,"`` This is not a Lehman, '' he said, referring..."


In [4]:
mysentences = df1['sentences'].tolist()
print(len(mysentences))

275


In [11]:
# create a csv file
fields = ['id', 'sentence', 'prompt', 'LLM_response']
filename = 'TimeML_TEST_zero_shot_round2.csv'
# writing to csv file
with open(filename, 'w') as csvfile:
    # creating a csv writer object
    csvwriter = csv.writer(csvfile)
    # writing the fields
    csvwriter.writerow(fields)

In [12]:
df = pd.read_csv('TimeML_TEST_zero_shot_round2.csv')

In [13]:
df

,id,sentence,prompt,LLM_response


In [14]:
def outputs(sentences):
    for sentence in sentences:
        ids = id(sentence)
        sentence = sentence
        prompt = f"""Given the sentence between the brackets <{sentence}>, annotate each token of the sentence, including punctuation marks, as an event trigger with the tag <EVENT> or as a non-event trigger with the tag <O>.
Return exclusively the annotated results token by token. Provide each token without brackets followed by its associated annotation tag in the same line. Provide each subsequent token next to its associated annotation tag in a new line.
Do not add any extra introduction, notes or explanation.
"""
        output = prompt_model(prompt)
        df.loc[len(df.index)] = [ids, sentence, prompt, output]
    return

In [15]:
outputs(mysentences)

In [16]:
df

,id,sentence,prompt,LLM_response
0,140128563332016,"A Cyprus exit from the euro union, if it come...",Given the sentence between the brackets < A Cy...,A <O>\nCyprus <O>\nexit <O>\nfrom <O>\nthe <O>...
1,140128563041488,"And for European unity and diplomacy, the Cypr...",Given the sentence between the brackets <And f...,And <EVENT>\nfor <O>\nEuropean <O>\nunity <O>\...
2,140128562949760,"ut for the broader financial system in Europe,...",Given the sentence between the brackets <ut fo...,ut <O>\nfor <O>\nthe <O>\nbroader <O>\nfinanci...
3,140128563272656,"And that, economists and investors contend, is...",Given the sentence between the brackets <And t...,And <EVENT>\nthat <O>\neconomists <EVENT>\nand...
4,140128563331792,Greece may well have been too big to fail last...,Given the sentence between the brackets <Greec...,Greece <O>\nmay <O>\nwell <O>\nhave <O>\nbeen ...
...,...,...,...,...
270,140128563398576,Memorial says inspectors returned to its Mosco...,Given the sentence between the brackets <Memor...,Memorial <O>\nsays <O>\ninspectors <EVENT>\nre...
271,140128564836208,A statement on the Memorial website said the i...,Given the sentence between the brackets <A sta...,A statement <O>\non <O>\nthe <O>\nMemorial <O>...
272,140128563337168,"Memorial director Arseny Roginsky, quoted by t...",Given the sentence between the brackets <Memor...,Memorial director <EVENT>\nArseny <O>\nRoginsk...
273,140128564525488,He insisted that the NGO law `` will not chang...,Given the sentence between the brackets <He in...,He <EVENT>\ninsisted <EVENT>\nthat <O>\nthe <O...


In [17]:
df.head(10)

,id,sentence,prompt,LLM_response
0,140128563332016,"A Cyprus exit from the euro union, if it come...",Given the sentence between the brackets < A Cy...,A <O>\nCyprus <O>\nexit <O>\nfrom <O>\nthe <O>...
1,140128563041488,"And for European unity and diplomacy, the Cypr...",Given the sentence between the brackets <And f...,And <EVENT>\nfor <O>\nEuropean <O>\nunity <O>\...
2,140128562949760,"ut for the broader financial system in Europe,...",Given the sentence between the brackets <ut fo...,ut <O>\nfor <O>\nthe <O>\nbroader <O>\nfinanci...
3,140128563272656,"And that, economists and investors contend, is...",Given the sentence between the brackets <And t...,And <EVENT>\nthat <O>\neconomists <EVENT>\nand...
4,140128563331792,Greece may well have been too big to fail last...,Given the sentence between the brackets <Greec...,Greece <O>\nmay <O>\nwell <O>\nhave <O>\nbeen ...
5,140128564755744,"From a financial standpoint, what is most note...",Given the sentence between the brackets <From ...,From <EVENT>\na <O>\nfinancial <O>\nstandpoint...
6,140128564755984,"Only Ireland, still struggling to recover from...",Given the sentence between the brackets <Only ...,Only <O>\nIreland <O>\nstill <O>\nstruggling <...
7,140128567533872,As debts in Europe mount in inverse proportion...,Given the sentence between the brackets <As de...,As <O> \ndebts <O> \nin <O> \nEurope <O> \nmou...
8,140128564756224,`` There have been too many bailouts in Europe...,Given the sentence between the brackets <`` Th...,There\n<EVENT>\nhave\n<EVENT>\nbeen\n<EVENT>\n...
9,140128564014576,"`` This is not a Lehman, '' he said, referring...",Given the sentence between the brackets <`` Th...,This\n<EVENT>\nis\n<O>\nnot\n<O>\na\n<O>\nLehm...


In [18]:
df.to_csv('TimeML_TEST_zero_shot_round2_withtabs.csv', sep = "\t")

In [19]:
df.to_csv('TimeML_TEST_zero_shot_round2_withcommas.csv')

In [20]:
df2 = pd.read_csv('TimeML_TEST_zero_shot_round2_withtabs.csv', sep = "\t")
df2.head()

,Unnamed: 0,id,sentence,prompt,LLM_response
0,0,140128563332016,"A Cyprus exit from the euro union, if it come...",Given the sentence between the brackets < A Cy...,A <O>\nCyprus <O>\nexit <O>\nfrom <O>\nthe <O>...
1,1,140128563041488,"And for European unity and diplomacy, the Cypr...",Given the sentence between the brackets <And f...,And <EVENT>\nfor <O>\nEuropean <O>\nunity <O>\...
2,2,140128562949760,"ut for the broader financial system in Europe,...",Given the sentence between the brackets <ut fo...,ut <O>\nfor <O>\nthe <O>\nbroader <O>\nfinanci...
3,3,140128563272656,"And that, economists and investors contend, is...",Given the sentence between the brackets <And t...,And <EVENT>\nthat <O>\neconomists <EVENT>\nand...
4,4,140128563331792,Greece may well have been too big to fail last...,Given the sentence between the brackets <Greec...,Greece <O>\nmay <O>\nwell <O>\nhave <O>\nbeen ...


In [2]:
df = pd.read_csv('TimeML_TEST_zero_shot_round2_withcommas.csv')
df.head()

,Unnamed: 0,id,sentence,prompt,LLM_response
0,0,140128563332016,"A Cyprus exit from the euro union, if it come...",Given the sentence between the brackets < A Cy...,A <O>\nCyprus <O>\nexit <O>\nfrom <O>\nthe <O>...
1,1,140128563041488,"And for European unity and diplomacy, the Cypr...",Given the sentence between the brackets <And f...,And <EVENT>\nfor <O>\nEuropean <O>\nunity <O>\...
2,2,140128562949760,"ut for the broader financial system in Europe,...",Given the sentence between the brackets <ut fo...,ut <O>\nfor <O>\nthe <O>\nbroader <O>\nfinanci...
3,3,140128563272656,"And that, economists and investors contend, is...",Given the sentence between the brackets <And t...,And <EVENT>\nthat <O>\neconomists <EVENT>\nand...
4,4,140128563331792,Greece may well have been too big to fail last...,Given the sentence between the brackets <Greec...,Greece <O>\nmay <O>\nwell <O>\nhave <O>\nbeen ...
